# OPIT classification with Random Forest (RF)

### Classification with settings from parameter testing for RF algorithm. 
### Using Google Earth Engine Python API and NICFI Normalized Analytic Basemap from December 2015

Author: Finn Geiger\
Date: April 6th 2023\
Contact:
- https://github.com/finn-geiger
- https://www.linkedin.com/in/finn-geiger-b1329a20b/

### 1 Import and setup
#### 1.1 Importing the required libraries and packages

In [32]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import geemap
import ee
import os
import time
import pandas as pd
from tabulate import tabulate
#%pip install tabulate


The following classes and landcover IDs will be used:

In [33]:
info = {'Class name': ['Informal', 'Formal', 'Industrial', 'Roads', 'Vacant land', 'Vegetation', 'Water-bodies'],
        'landcover ID': [1, 2, 3, 4, 5, 6, 7]}

print(tabulate(info, headers='keys', tablefmt='fancy_grid'))

╒══════════════╤════════════════╕
│ Class name   │   landcover ID │
╞══════════════╪════════════════╡
│ Informal     │              1 │
├──────────────┼────────────────┤
│ Formal       │              2 │
├──────────────┼────────────────┤
│ Industrial   │              3 │
├──────────────┼────────────────┤
│ Roads        │              4 │
├──────────────┼────────────────┤
│ Vacant land  │              5 │
├──────────────┼────────────────┤
│ Vegetation   │              6 │
├──────────────┼────────────────┤
│ Water-bodies │              7 │
╘══════════════╧════════════════╛


##### When first using the GEE Python API the user must authenticate and initialize the environment by using the following two lines of codes:

In [34]:
#ee.Authenticate() 
#ee.Initialize()

In [35]:
# creating the map
Map = geemap.Map()

# loading the interactive map
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

#### 1.2 Importing the datasets from GEE assets and data catalog and clipping the basemap to the AOI

In [36]:
# Loading the Base scene
nicfi = ee.ImageCollection('projects/planet-nicfi/assets/basemaps/africa')

# Filter basemaps by date and get the first image from filtered results
basemap_2015_12   = nicfi.filter(ee.Filter.date('2015-12-01','2016-01-01')).first()


# Visualizing the scene
vis_params = {"bands":["R","G","B"],"min":64,"max":5454,"gamma":1.8}

# Adding the basemap to the map
Map.centerObject(basemap_2015_12, 4)
Map.addLayer(basemap_2015_12, vis_params, '2015-12 mosaic')

In [37]:
# Loading the AOI and Masking the base scene
vis_params_aoi = {'color': 'blue'}
aoi_windhoek = ee.FeatureCollection('users/s85315/masterthesis/Study_Area_Windhoek')

# Adding the AOI to the map
Map.addLayer(aoi_windhoek, vis_params_aoi, 'AOI')
Map.centerObject(aoi_windhoek, 12)

In [38]:
# clipping the clipped_TOI1_2015 to the AOI
clipped_TOI1_2015 = basemap_2015_12.clipToCollection(aoi_windhoek)
Map.addLayer(clipped_TOI1_2015, vis_params, 'clipped')

In [39]:
# importing training data samples and adding them to the map
# Transforming the Geometries into FeatureCollections and applying properties
training_samples = ee.FeatureCollection('users/s85315/masterthesis/TrainingSamples/ChangeDetection/TS_TOI1_2015_RPoints')

# creating the visualization parameters

Map.addLayer(training_samples, {}, 'Training Samples for 2015')

### 2 Classification with RF

#### 2.1 Applying training samples on the base scene

In [40]:
# adding the training samples to the basescene
training = clipped_TOI1_2015.sampleRegions(**{
    'collection': training_samples, 
    'properties': ['landcover'], 
    'scale': 4.77,
    'projection': 'EPSG:32733'
})

#### 2.2 Configuration and creation of the empty RF classifier

In [41]:
# creating variable for parameter settings
# Mtry will be set to default
RF_param = 550

classifier_params = {
              'numberOfTrees':RF_param, # 	Ntree; The number of decision trees to create.
              'variablesPerSplit':None, # Mtry; The number of variables per split. If unspecified, uses the square root of the number of variables.
              'minLeafPopulation':1, # smallest sample size possible per leaf
              'bagFraction':0.5, #The fraction of input to bag per tree.
              'maxNodes':None, # the number of individual decision tree models
              'seed': 0}  # The randomization seed.


# creating the classifier using RF
classifier = ee.Classifier.smileRandomForest(**classifier_params).train(**{
  'features': training,  
  'classProperty': 'landcover', 
  'inputProperties': clipped_TOI1_2015.bandNames()
})

#### 2.3 classifying the basescene and visualizing the product

In [42]:
# classifying the basescene using the created classifier
classified_TOI1_2015 = clipped_TOI1_2015.classify(classifier)

# creating the visualization parameters
palette = ['c43c39', 'e5b636', '2f2f2f', 'aaaaaa', 'b08e7a', '85b66f', 'a5bfdd']
vis_params_classified = {'min': 1, 'max': 7, 'palette': palette}


Map.addLayer(classified_TOI1_2015, vis_params_classified, 'classified basescene')


#### 2.4 Exporting the results

##### 2.4.1 Exporting to Google Drive

In [43]:
# converting the FeatureCollection to Geometry for export
aoi_geom = aoi_windhoek.geometry()

# exporting to Google drive with GEE API
export = ee.batch.Export.image.toDrive(**{
    'image': classified_TOI1_2015,
    'description': 'classified_TOI1_RF', # TODO: change name here
    'folder': 'masterthesis/change_detection_results', # TODO: change name here
    'scale': 4.77,
    'region': aoi_geom,
    'crs': 'EPSG:32733'
})

# starting the process
export.start()

# tracking the process
while export.active():
  print('Polling for task (id: {}).'.format(export.id))
  time.sleep(5)

Polling for task (id: YKMLATNJIN4Z5TOCICAAPMEG).
Polling for task (id: YKMLATNJIN4Z5TOCICAAPMEG).
Polling for task (id: YKMLATNJIN4Z5TOCICAAPMEG).
Polling for task (id: YKMLATNJIN4Z5TOCICAAPMEG).
Polling for task (id: YKMLATNJIN4Z5TOCICAAPMEG).
Polling for task (id: YKMLATNJIN4Z5TOCICAAPMEG).
Polling for task (id: YKMLATNJIN4Z5TOCICAAPMEG).
Polling for task (id: YKMLATNJIN4Z5TOCICAAPMEG).
Polling for task (id: YKMLATNJIN4Z5TOCICAAPMEG).
Polling for task (id: YKMLATNJIN4Z5TOCICAAPMEG).
Polling for task (id: YKMLATNJIN4Z5TOCICAAPMEG).
Polling for task (id: YKMLATNJIN4Z5TOCICAAPMEG).
Polling for task (id: YKMLATNJIN4Z5TOCICAAPMEG).
Polling for task (id: YKMLATNJIN4Z5TOCICAAPMEG).
Polling for task (id: YKMLATNJIN4Z5TOCICAAPMEG).
Polling for task (id: YKMLATNJIN4Z5TOCICAAPMEG).
Polling for task (id: YKMLATNJIN4Z5TOCICAAPMEG).
Polling for task (id: YKMLATNJIN4Z5TOCICAAPMEG).
Polling for task (id: YKMLATNJIN4Z5TOCICAAPMEG).
Polling for task (id: YKMLATNJIN4Z5TOCICAAPMEG).
Polling for task (id

##### 2.4.2 Exporting to Asset

In [44]:
# # exporting to Google Asset
# export = ee.batch.Export.image.toAsset(**{
#   'image': classified_TOI1_2015,
#   'description': 'Export classified map',
#   'assetId': 'users/s85315/masterthesis/change_detection_results/classified_TOI1_RF', # TODO: change name here
#   'scale': 4.77,
#   'region': aoi_geom,
#   'crs': 'EPSG:32733'
# })

# # starting the process
# export.start()

# # tracking the process
# while export.active():
#   print('Polling for task (id: {}).'.format(export.id))
#   time.sleep(5)

### 3 Calulation of the class size from pixel size

In [45]:
# calculating pixel size
area_classified_2015 = ee.Image.pixelArea().addBands(classified_TOI1_2015)

# calculating the area size per class
areas_2015 = area_classified_2015.reduceRegion(**{
    'reducer': ee.Reducer.sum().group(**{
    'groupField': 1,
    'groupName': 'landcover',
    }),
    'geometry': aoi_geom, 
    'scale': 4.77
    })

# selecting the size per class
class_areas_2015 = ee.List(areas_2015.get('groups'))
list_dict_2015 = class_areas_2015.getInfo()

# creating a dataframe for easier data handling
df_areas_2015 = pd.DataFrame(class_areas_2015.getInfo())
df_areas_2015['sum'] = df_areas_2015['sum'].astype(int)
print(df_areas_2015['sum'].sum())
df_areas_2015.to_csv('./data/class_size_2015.csv', sep=";", index=False)

print(tabulate(df_areas_2015, tablefmt='fancy_grid', headers=['Landcover', 'Area size [m²]'], showindex=False, floatfmt=".0f"))

81697971
╒═════════════╤══════════════════╕
│   Landcover │   Area size [m²] │
╞═════════════╪══════════════════╡
│           1 │          8064411 │
├─────────────┼──────────────────┤
│           2 │          7627039 │
├─────────────┼──────────────────┤
│           3 │          2796382 │
├─────────────┼──────────────────┤
│           4 │          6263676 │
├─────────────┼──────────────────┤
│           5 │         40621551 │
├─────────────┼──────────────────┤
│           6 │         15399511 │
├─────────────┼──────────────────┤
│           7 │           925401 │
╘═════════════╧══════════════════╛


### 4 Accuracy assessment

#### 4.1 Importing validation samples from GEE Assets

In [55]:
# Importing merged validation samples for 2015
validation_samples = ee.FeatureCollection('users/s85315/masterthesis/ValidationSamples/ChangeDetection/VS_TOI1_2015_RPoints')
#validation_samples = ee.FeatureCollection('users/s85315/masterthesis/ValidationSamples/ChangeDetection/VS_TOI1_2015_RPoints_test')

Map.addLayer(validation_samples, {}, 'VS Samples')

#### 4.2 Applying the validation samples to the basescene

In [47]:
# applying the validation samples to the classified map
validation = classified_TOI1_2015.sampleRegions(**{
  'collection': validation_samples,
  'properties': ['landcover'],
  'tileScale': 16,
  'scale': 4.77,
  'projection': 'EPSG:32733'
})

#### 4.3 Generating the error matrix and printing information

In [48]:
#order = training.aggregate_array('landcover').distinct().sort()

#basescene_error_matrix = validation.errorMatrix(actual='validation', predicted='classification', order=order)
basescene_error_matrix = validation.errorMatrix('classification', 'landcover')
basescene_error_matrix_test = validation.errorMatrix('classification', 'landcover')

# printing statistics
print('Confusion Matrix', basescene_error_matrix.getInfo())
print('Overall Accuracy', basescene_error_matrix.accuracy().getInfo())
print('Producers Accuracy', basescene_error_matrix.producersAccuracy().getInfo())
print('Consumers Accuracy', basescene_error_matrix.consumersAccuracy().getInfo())
print('Kappa', basescene_error_matrix.kappa().getInfo())

print(basescene_error_matrix.array().getInfo())

Confusion Matrix [[0, 0, 0, 0, 0, 0, 0, 0], [0, 26, 8, 4, 0, 2, 2, 0], [0, 7, 24, 12, 2, 0, 0, 0], [0, 1, 8, 16, 1, 0, 0, 0], [0, 6, 3, 2, 36, 5, 0, 0], [0, 5, 0, 0, 1, 77, 6, 0], [0, 0, 0, 1, 2, 16, 45, 1], [0, 0, 0, 0, 0, 0, 0, 31]]
Overall Accuracy 0.7285714285714285
Producers Accuracy [[0], [0.6190476190476191], [0.5333333333333333], [0.6153846153846154], [0.6923076923076923], [0.8651685393258427], [0.6923076923076923], [1]]
Consumers Accuracy [[0, 0.5777777777777777, 0.5581395348837209, 0.45714285714285713, 0.8571428571428571, 0.77, 0.8490566037735849, 0.96875]]
Kappa 0.6747975431321154
[[0, 0, 0, 0, 0, 0, 0, 0], [0, 26, 8, 4, 0, 2, 2, 0], [0, 7, 24, 12, 2, 0, 0, 0], [0, 1, 8, 16, 1, 0, 0, 0], [0, 6, 3, 2, 36, 5, 0, 0], [0, 5, 0, 0, 1, 77, 6, 0], [0, 0, 0, 1, 2, 16, 45, 1], [0, 0, 0, 0, 0, 0, 0, 31]]


##### 4.3.1 Visualizing the error matrix

In [54]:
# creating a Pandas Dataframe for the error matrix
error_matrix = basescene_error_matrix.getInfo()
df_error_matrix = pd.DataFrame(error_matrix)

# deleting the first row and column since GEE add's a class with the landcover ID 0 by default.
df_error_matrix.columns = ['not used','Informal', 'Formal', 'Industrial', 'Roads', 'Vacant land', 'Vegetation', 'Water-bodies']
df_error_matrix = df_error_matrix.drop(df_error_matrix.columns[0],axis=1)
df_error_matrix.drop(index=df_error_matrix.index[0], axis=0, inplace=True)

# calculating row and column sum of points
column_total = df_error_matrix.sum()
column_total.name = 'Total'
df_error_matrix.loc[8] = column_total
df_error_matrix['Total'] = df_error_matrix.sum(axis=1)

header_error_matrix = ['Informal', 'Formal', 'Industrial', 'Roads', 'Vacant land', 'Vegetation', 'Water-bodies', 'Total']
df_error_matrix['Names'] = header_error_matrix
df_error_matrix = df_error_matrix.set_index('Names')


print(tabulate(df_error_matrix, headers=header_error_matrix, tablefmt='fancy_grid', showindex=header_error_matrix))
df_error_matrix.to_csv("./data/accuracies/CD_Error_Matrix_TOI1.csv", sep=';', index=True)

╒══════════════╤════════════╤══════════╤══════════════╤═════════╤═══════════════╤══════════════╤════════════════╤═════════╕
│              │   Informal │   Formal │   Industrial │   Roads │   Vacant land │   Vegetation │   Water-bodies │   Total │
╞══════════════╪════════════╪══════════╪══════════════╪═════════╪═══════════════╪══════════════╪════════════════╪═════════╡
│ Informal     │         26 │        8 │            4 │       0 │             2 │            2 │              0 │      42 │
├──────────────┼────────────┼──────────┼──────────────┼─────────┼───────────────┼──────────────┼────────────────┼─────────┤
│ Formal       │          7 │       24 │           12 │       2 │             0 │            0 │              0 │      45 │
├──────────────┼────────────┼──────────┼──────────────┼─────────┼───────────────┼──────────────┼────────────────┼─────────┤
│ Industrial   │          1 │        8 │           16 │       1 │             0 │            0 │              0 │      26 │
├───────

##### 4.3.2 Producer's and consumer's accuracy

In [53]:
# creating the lists 
producers = basescene_error_matrix.producersAccuracy().getInfo()
df_producers = pd.DataFrame(producers)
df_producers.drop(index=df_producers.index[0], axis=0, inplace=True)

class_names = ['Informal', 'Formal', 'Industrial', 'Roads', 'Vacant land', 'Vegetation', 'Water-bodies']
df_producers['class names'] = class_names
df_producers.columns = ["Producer Accuracy", "Class name"]
df_producers['Producer Accuracy'] = df_producers['Producer Accuracy'].multiply(100).round(2)

# creating a dataframe from the list of consumer accuracies and remove landcover ID 0
consumers = basescene_error_matrix.consumersAccuracy().getInfo()
df_consumers = pd.DataFrame(consumers)
df_consumers = df_consumers.drop(df_consumers.columns[0],axis=1)
df_consumers.columns = class_names

# reshaping the dataframe from wide to long format:
df_consumers_long = pd.melt(df_consumers, var_name='Class name', value_name="Consumer Accuracy")
df_consumers_long = df_consumers_long[['Consumer Accuracy', 'Class name']]
df_consumers_long['Consumer Accuracy'] = df_consumers_long['Consumer Accuracy'].multiply(100).round(2)

# merging both dataframes
df_all = pd.merge(df_producers, df_consumers_long, on='Class name')
new_cols = ['Class name', 'Producer Accuracy', 'Consumer Accuracy']
df_all = df_all[new_cols]


print(tabulate(df_all, headers=["Class name", "Producer's Accuracy [%]", "Consumers's Accuracy [%]"], tablefmt='fancy_grid',  showindex=False))
df_all.to_csv("./data/accuracies/CD_Consumers_Producers_Accuracy_TOI1.csv", sep=";", index=False)

╒══════════════╤═══════════════════════════╤════════════════════════════╕
│ Class name   │   Producer's Accuracy [%] │   Consumers's Accuracy [%] │
╞══════════════╪═══════════════════════════╪════════════════════════════╡
│ Informal     │                     61.9  │                      57.78 │
├──────────────┼───────────────────────────┼────────────────────────────┤
│ Formal       │                     53.33 │                      55.81 │
├──────────────┼───────────────────────────┼────────────────────────────┤
│ Industrial   │                     61.54 │                      45.71 │
├──────────────┼───────────────────────────┼────────────────────────────┤
│ Roads        │                     69.23 │                      85.71 │
├──────────────┼───────────────────────────┼────────────────────────────┤
│ Vacant land  │                     86.52 │                      77    │
├──────────────┼───────────────────────────┼────────────────────────────┤
│ Vegetation   │                     6

##### 4.3.3 Overall Accuracy and Kappa Coefficent

In [52]:
# defining the variables
overall_accuracy = basescene_error_matrix.accuracy().getInfo()
overall_print = str(round(overall_accuracy * 100, 2))
kappa = basescene_error_matrix.kappa().getInfo()

df_overall_kappa = pd.DataFrame()

# printing out vaLues
print("\033[1m" + "Overall Accuracy " + overall_print + " %" + "\033[0m")
print("\033[1m" + "Kappa coefficent " + str(round(kappa, 2)) + "\033[0m")

Overall Accuracy 72.86 %
Kappa coefficent 0.67


##### Resources for code snippets

https://colab.research.google.com/github/csaybar/EEwPython/blob/dev/10_Export.ipynb \
https://worldbank.github.io/OpenNightLights/tutorials/mod6_6_RF_classifier.html \
https://towardsdatascience.com/how-to-easily-create-tables-in-python-2eaea447d8fd \
https://developers.google.com/earth-engine/apidocs/ee-classifier-smilerandomforest